In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
from torchvision.models import resnet18, ResNet18_Weights
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from scipy.stats import rankdata

# Use GPU for PyTorch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# --- AUTO-FIND PATHS ---
TRAIN_DIR = ""
TEST_DIR = ""
for root, dirs, files in os.walk('/kaggle/input'):
    if "training_videos" in dirs:
        TRAIN_DIR = os.path.join(root, 'training_videos')
    if "testing_videos" in dirs:
        TEST_DIR = os.path.join(root, 'testing_videos')

if TRAIN_DIR and TEST_DIR:
    print(f"✅ Found Dataset:\nTrain: {TRAIN_DIR}\nTest:  {TEST_DIR}")
else:
    print("❌ ERROR: Dataset paths not found!")


2026-01-05 20:55:22.930934: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767646523.124280      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767646523.177163      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767646523.641050      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767646523.641089      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767646523.641091      55 computation_placer.cc:177] computation placer alr

Using device: cuda
✅ Found Dataset:
Train: /kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/training_videos
Test:  /kaggle/input/pixel-play-26/Avenue_Corrupted-20251221T112159Z-3-001/Avenue_Corrupted/Dataset/testing_videos


In [2]:
# --- BUILD 2D EXTRACTOR ---
print("Loading ResNet18 (ImageNet)...")
weights = ResNet18_Weights.IMAGENET1K_V1
model_2d = resnet18(weights=weights).to(device)
# Remove classification layer to get 512-dim features
model_2d.fc = nn.Identity()
model_2d.eval()

# Standard ImageNet Transform
transform_2d = weights.transforms()

def extract_features_from_folder(folder_path):
    """
    Reads all images in a folder and returns their 512-dim features.
    Returns: (N, 512) array, List of paths
    """
    image_paths = sorted(glob.glob(os.path.join(folder_path, "*")))
    if len(image_paths) == 0: return None, None
    
    frames = []
    for p in image_paths:
        img = cv2.imread(p)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(img)
            
    if len(frames) == 0: return None, None
    
    # Process in batches to avoid OOM
    batch_size = 128
    feats = []
    
    for i in range(0, len(frames), batch_size):
        batch_imgs = frames[i:i+batch_size]
        
        # Preprocess: List of Arrays -> Stack -> To Tensor -> Transform
        # Note: transform_2d expects (C,H,W), but cv2 gives (H,W,C). 
        # We must permute BEFORE transform.
        batch_tensor = torch.stack([
            transform_2d(torch.from_numpy(img).permute(2,0,1)) 
            for img in batch_imgs
        ]).to(device)
        
        with torch.no_grad():
            f = model_2d(batch_tensor)
            feats.append(f.cpu().numpy())
            
    return np.vstack(feats), image_paths

print("✅ 2D Extractor Ready!")


Loading ResNet18 (ImageNet)...
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 238MB/s]


✅ 2D Extractor Ready!


In [3]:
# --- 1. EXTRACT TRAIN FEATURES ---
print("🚀 Phase 1: Extracting 2D Features from Training Videos...")
train_folders = sorted(glob.glob(os.path.join(TRAIN_DIR, "*")))

all_train_feats = []

for folder in tqdm(train_folders):
    f, _ = extract_features_from_folder(folder)
    if f is not None:
        all_train_feats.append(f)

# Stack all features: (Total_Train_Frames, 512)
X_train = np.vstack(all_train_feats)
print(f"Training Data Shape: {X_train.shape}")

# --- 2. BUILD DAE MODEL ---
# Input dim is 512 (ResNet18 output)
INPUT_DIM = 512 

def build_dae(input_dim):
    inputs = layers.Input(shape=(input_dim,))
    
    # Noise
    x = layers.GaussianNoise(0.1)(inputs)
    
    # Encoder
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Dense(64)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Bottleneck
    code = layers.Dense(32)(x)
    x = layers.BatchNormalization()(code)
    code = layers.Activation('relu')(x)
    
    # Decoder
    x = layers.Dense(64)(code)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    x = layers.Dense(256)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Output
    outputs = layers.Dense(input_dim, activation='linear')(x)
    
    model = models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='mse')
    return model

dae = build_dae(INPUT_DIM)

# --- 3. TRAIN ---
print("🚀 Phase 2: Training DAE...")
dae.fit(
    X_train, X_train,
    epochs=50,
    batch_size=128,
    shuffle=True,
    verbose=1
)


🚀 Phase 1: Extracting 2D Features from Training Videos...


100%|██████████| 16/16 [01:45<00:00,  6.58s/it]
I0000 00:00:1767646686.560303      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14161 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Training Data Shape: (9204, 512)
🚀 Phase 2: Training DAE...
Epoch 1/50


I0000 00:00:1767646691.037352     128 service.cc:152] XLA service 0x7e425c008390 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1767646691.037385     128 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1767646691.635006     128 cuda_dnn.cc:529] Loaded cuDNN version 91002


61/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4855

I0000 00:00:1767646693.640013     128 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


72/72 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - loss: 0.4420
Epoch 2/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0545
Epoch 3/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0487
Epoch 4/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0448
Epoch 5/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0421
Epoch 6/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0404
Epoch 7/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0388
Epoch 8/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0368
Epoch 9/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0362
Epoch 10/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0345
Epoch 11/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0341
Epoch 12/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0332
Epoch 13/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0322
Epoch 14/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0305
Epoch 15/50
72/72 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0305
Epoch 16/50
72/72 ━━━━━━━━━━━

In [4]:
# --- INFERENCE WITH TTA (Per-Frame Minimum Error) ---
print("🚀 Phase 3: Inference on Test Videos (With Frame-Wise Rotation Check)...")
test_folders = sorted(glob.glob(os.path.join(TEST_DIR, "*")))

all_ids = []
all_errors = []

for folder in tqdm(test_folders):
    # 1. READ IMAGES
    image_paths = sorted(glob.glob(os.path.join(folder, "*")))
    if len(image_paths) == 0: continue
        
    frames = []
    for p in image_paths:
        img = cv2.imread(p)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            frames.append(img)
            
    if len(frames) == 0: continue
    
    # 2. PROCESS BATCHES
    batch_size = 128 # Larger batch since 2D is lighter
    folder_errors = []
    
    for i in range(0, len(frames), batch_size):
        batch_imgs = frames[i:i+batch_size]
        
        # A. Normal Tensor (Upright)
        t_norm = torch.stack([
            transform_2d(torch.from_numpy(img).permute(2,0,1)) 
            for img in batch_imgs
        ]).to(device)
        
        # B. Rotated Tensor (Upside Down / 180)
        # Flip Height (dim 2) and Width (dim 3)
        t_rot = torch.flip(t_norm, [2, 3])
        
        with torch.no_grad():
            # Extract features for both orientations
            f_norm = model_2d(t_norm).cpu().numpy()
            f_rot = model_2d(t_rot).cpu().numpy()
            
        # C. Reconstruct Both
        recon_norm = dae.predict(f_norm, verbose=0)
        mse_norm = np.mean((f_norm - recon_norm)**2, axis=1)
        
        recon_rot = dae.predict(f_rot, verbose=0)
        mse_rot = np.mean((f_rot - recon_rot)**2, axis=1)
        
        # D. MINIMUM LOGIC (The "Fit")
        # For each frame INDEPENDENTLY:
        # If frame is Upright -> mse_norm is low -> picked.
        # If frame is Inverted -> mse_rot is low -> picked.
        # If Anomaly -> Both are high.
        batch_mse = np.minimum(mse_norm, mse_rot)
        
        folder_errors.extend(batch_mse)

    # 3. Store IDs (Paper Format)
    for p, score in zip(image_paths, folder_errors):
        parts = p.split(os.sep)
        # Extract folder number and file number
        f_num = int(re.findall(r'\d+', parts[-2])[-1])
        i_num = int(re.findall(r'\d+', parts[-1])[-1])
        clean_id = f"{f_num}_{i_num}"
        
        all_ids.append(clean_id)
        all_errors.append(score)

# --- SAVE ---
submission_df = pd.DataFrame({
    'Id': all_ids,
    # Standard Rank Normalization only (No PSNR/Gaussian as requested)
    'Predicted': rankdata(all_errors) / len(all_errors)
})

# Sort to ensure correct order
submission_df[['Video', 'Frame']] = submission_df['Id'].str.split('_', expand=True).astype(int)
submission_df = submission_df.sort_values(['Video', 'Frame'])
submission_df = submission_df.drop(columns=['Video', 'Frame'])

submission_df.to_csv('submission_paper_semantic_optimized.csv', index=False)
print("🚀 Optimized Submission Saved!")
print(submission_df.head())


🚀 Phase 3: Inference on Test Videos (With Frame-Wise Rotation Check)...


100%|██████████| 21/21 [02:43<00:00,  7.78s/it]

🚀 Optimized Submission Saved!
      Id  Predicted
0  1_939   0.146421
1  1_940   0.141295
2  1_941   0.329147
3  1_942   0.542969
4  1_943   0.522211


In [5]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d

# 1. Load your best baseline
df = pd.read_csv('/kaggle/working/submission_paper_semantic_optimized.csv')
df[['Video', 'Frame']] = df['Id'].str.split('_', expand=True).astype(int)
df = df.sort_values(['Video', 'Frame'])

# 2. MSE -> PSNR Conversion
# Peak Signal-to-Noise Ratio focuses on the reconstruction quality
epsilon = 1e-10
df['PSNR'] = 10 * np.log10(1.0 / (df['Predicted'] + epsilon))

# 3. Choose your Normalization Strategy
# If Per-Video lowered your score before, try 'GLOBAL'
STRATEGY = 'GLOBAL' 

if STRATEGY == 'PER_VIDEO':
    def norm_func(x):
        return 1 - ((x - x.min()) / (x.max() - x.min() + epsilon))
    df['Anomaly_Score'] = df.groupby('Video')['PSNR'].transform(norm_func)
else:
    # Global normalization preserves the 'intensity' difference between videos
    p_min, p_max = df['PSNR'].min(), df['PSNR'].max()
    df['Anomaly_Score'] = 1 - ((df['PSNR'] - p_min) / (p_max - p_min + epsilon))

# 4. Gaussian Smoothing (More precise than Moving Average)
# sigma=1.0 or 1.5 is the 'sweet spot' for 24-30fps video
df['Smoothed'] = df.groupby('Video')['Anomaly_Score'].transform(
    lambda x: gaussian_filter1d(x, sigma=4)
)

# 5. Save the two best candidates
df[['Id', 'Anomaly_Score']].rename(columns={'Anomaly_Score': 'Predicted'}).to_csv('sub_psnr_only.csv', index=False)
df[['Id', 'Smoothed']].rename(columns={'Smoothed': 'Predicted'}).to_csv('sub_psnr_gaussian.csv', index=False)

print("Generated 'sub_psnr_only.csv' and 'sub_psnr_gaussian.csv'")

Generated 'sub_psnr_only.csv' and 'sub_psnr_gaussian.csv'
